In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
changeupDL = pd.read_csv('DraftLeagueChangeup.csv')
changeupMLB = pd.read_csv('MLBchangeup.csv')

In [3]:
changeupDL = changeupDL.astype(str)
changeupMLB = changeupMLB.astype(str)

In [4]:
changeupDL['league'] = 'Draft League'
changeupMLB['league'] = 'MLB'

In [5]:
pitchersDL = list(set(changeupDL['pitcher']))

In [6]:
def combined_features(row):
    return row['avgvelo']+" "+row['avgspinrate']+" "+row['avghorzbreak']+" "+row['avgvertbreak']+" "+row['avgeffectvelo']

In [7]:
def get_index_from_pitcher(pitcher):
    return changeup[changeup.pitcher == pitcher].index[0]

In [8]:
def get_pitcher_from_index(index):
    return changeup[changeup.index == index].values[0][0]

In [9]:
mlb_draft_pitcher = []
mlb_comp = []
cos_value = []

for x in pitchersDL:
    
    mlb_draft_pitcher.append(x)
    
    try:

        changeup = pd.concat([changeupDL, changeupMLB])

        changeup["combined_features"] = changeup.apply(combined_features, axis =1)

        cv = CountVectorizer()
        count_matrix = cv.fit_transform(changeup["combined_features"])

        cosine_sim = cosine_similarity(count_matrix)

        pitcher_like = x
        changeup = changeup[(changeup['pitcher'] == pitcher_like) | (changeup['league'] == 'MLB')]

        player_index = get_index_from_pitcher(pitcher_like)

        similar_pitchers = list(enumerate(cosine_sim[player_index]))

        sorted_similar_pitchers = sorted(similar_pitchers, key=lambda x:x[1], reverse=True)
        
        mlb_comp.append(get_pitcher_from_index(sorted_similar_pitchers[1][0]))
        
        cos_value.append(sorted_similar_pitchers[1][1])
                
    except IndexError:
        mlb_comp.append('No Comp')
        cos_value.append('No Similarity')
        pass
    

In [10]:
cosine_comp = pd.DataFrame(list(zip(mlb_draft_pitcher, mlb_comp, cos_value)), columns=['MLB DL Pitcher', 'MLB Comp', "Cosine Value"])
cosine_comp

,MLB DL Pitcher,MLB Comp,Cosine Value
0,"Cherry, Derrick",No Comp,No Similarity
1,"Reeves, Cobe","Mercedes, Yermín",0.316228
2,"Dieter, Brandon","Sneed, Cy",0.33541
3,"Drake, Kohl","Zimmermann, Bruce",0.33541
4,"Camfield, Geoffrey","Valdez, Framber",0.3
...,...,...,...
115,"Sikorski, Easton","Mercedes, Yermín",0.316228
116,"Kush, Jacob","Mercedes, Yermín",0.333333
117,"Gonzalez, Joan","Herrera, Kelvin",0.258199
118,"Pratt, Davis","Fulmer, Carson",0.4


In [11]:
cosine_comp.to_csv('changeup_cosine_comp.csv', sep='\t')